# Module 12 Challenge
## Deliverable 2: Scrape and Analyze Mars Weather Data

In [3]:
# Import relevant libraries
from splinter import Browser
from bs4 import BeautifulSoup as soup
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
browser = Browser('chrome')

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 115
Current browser version is 119.0.6045.125 with binary path C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00007FF686D04A62+57106]
	(No symbol) [0x00007FF686C7CF52]
	(No symbol) [0x00007FF686B4E2CB]
	(No symbol) [0x00007FF686B79FDB]
	(No symbol) [0x00007FF686B74F8D]
	(No symbol) [0x00007FF686B730AA]
	(No symbol) [0x00007FF686BAE3F7]
	(No symbol) [0x00007FF686BADF60]
	(No symbol) [0x00007FF686BA6793]
	(No symbol) [0x00007FF686B7CE81]
	(No symbol) [0x00007FF686B7E064]
	GetHandleVerifier [0x00007FF686FB4222+2873042]
	GetHandleVerifier [0x00007FF687006590+3209792]
	GetHandleVerifier [0x00007FF686FFF3AF+3180639]
	GetHandleVerifier [0x00007FF686D95F25+652245]
	(No symbol) [0x00007FF686C88618]
	(No symbol) [0x00007FF686C847C4]
	(No symbol) [0x00007FF686C848BC]
	(No symbol) [0x00007FF686C74C33]
	BaseThreadInitThunk [0x0

### Step 1: Visit the Website

Use automated browsing to visit the [Mars Temperature Data Site](https://static.bc-edx.com/data/web/mars_facts/temperature.html). Inspect the page to identify which elements to scrape.

   > **Hint** To identify which elements to scrape, you might want to inspect the page by using Chrome DevTools to discover whether the table contains usable classes.


In [ ]:
# Visit the website
# https://static.bc-edx.com/data/web/mars_facts/temperature.html
url = "https://static.bc-edx.com/data/web/mars_facts/temperature.html"
browser.visit(url)

### Step 2: Scrape the Table

Create a Beautiful Soup object and use it to scrape the data in the HTML table.

Note that this can also be achieved by using the Pandas `read_html` function. However, use Beautiful Soup here to continue sharpening your web scraping skills.

In [ ]:
# Create a Beautiful Soup Object
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# Extract all rows of data
rows = soup.find_all('tr', class_="data-row")

### Step 3: Store the Data

Assemble the scraped data into a Pandas DataFrame. The columns should have the same headings as the table on the website. Here’s an explanation of the column headings:

* `id`: the identification number of a single transmission from the Curiosity rover
* `terrestrial_date`: the date on Earth
* `sol`: the number of elapsed sols (Martian days) since Curiosity landed on Mars
* `ls`: the solar longitude
* `month`: the Martian month
* `min_temp`: the minimum temperature, in Celsius, of a single Martian day (sol)
* `pressure`: The atmospheric pressure at Curiosity's location

In [ ]:
# Create an empty list
list_of_rows = []
# Loop through the scraped data to create a list of rows
for row in rows:
    td = row.find_all('td')
    row = [col.text for col in td]
    list_of_rows.append(row)

In [ ]:
# Create a Pandas DataFrame by using the list of rows and a list of the column names
df = pd.DataFrame(list_of_rows, columns = ["id", "terrestrial_date", "sol", "ls", "month", "min_temp", "pressure"])

In [ ]:
# Confirm DataFrame was created successfully
df.head()

### Step 4: Prepare Data for Analysis

Examine the data types that are currently associated with each column. If necessary, cast (or convert) the data to the appropriate `datetime`, `int`, or `float` data types.

  > **Hint** You can use the Pandas `astype` and `to_datetime` methods to accomplish this task.


In [ ]:
# Examine data type of each column
df.dtypes

In [ ]:
# Change data types for data analysis
df.terrestrial_date = pd.to_datetime(df.terrestrial_date)
df.sol = df.sol.astype('int')
df.ls = df.ls.astype('int')
df.month = df.month.astype('int')
df.min_temp = df.min_temp.astype('float')
df.pressure = df.pressure.astype('float')

In [ ]:
# Confirm type changes were successful by examining data types again
df.dtypes

### Step 5: Analyze the Data

Analyze your dataset by using Pandas functions to answer the following questions:

1. How many months exist on Mars?
2. How many Martian (and not Earth) days worth of data exist in the scraped dataset?
3. What are the coldest and the warmest months on Mars (at the location of Curiosity)? To answer this question:
    * Find the average the minimum daily temperature for all of the months.
    * Plot the results as a bar chart.
4. Which months have the lowest and the highest atmospheric pressure on Mars? To answer this question:
    * Find the average the daily atmospheric pressure of all the months.
    * Plot the results as a bar chart.
5. About how many terrestrial (Earth) days exist in a Martian year? To answer this question:
    * Consider how many days elapse on Earth in the time that Mars circles the Sun once.
    * Visually estimate the result by plotting the daily minimum temperature.


In [ ]:
# 1. How many months are there on Mars?
df["month"].value_counts().sort_index()

In [ ]:
# 2. How many Martian days' worth of data are there?
df.sol.nunique()

In [ ]:
# 3. What is the average low temperature by month?
min_temp_by_month = df.groupby('month')['min_temp'].mean()
print(min_temp_by_month)

In [ ]:
# Plot the average temperature by month
min_temp_by_month.plot(kind='bar')
plt.ylabel('Temperature in Celsius')
plt.show()

In [ ]:
# Identify the coldest and hottest months in Curiosity's location
min_temp_by_month.sort_values().plot(kind='bar')
plt.ylabel('Temperature in Celsius')
plt.show()

In [ ]:
# 4. Average pressure by Martian month
pressure_by_month = df.groupby('month')['pressure'].mean()
print(pressure_by_month)

In [ ]:
# Plot the average pressure by month
pressure_by_month.sort_values().plot(kind='bar')
plt.ylabel('Atmospheric Pressure')
plt.show()

In [ ]:
# 5. How many terrestrial (earth) days are there in a Martian year?
df.min_temp.plot()
plt.xlabel('Number of terrestrial days')
plt.ylabel('Minimum temperature')
plt.show()

On average, the third month has the coldest minimum temperature on Mars, and the eighth month is the warmest. But it is always very cold there in human terms!



Atmospheric pressure is, on average, lowest in the sixth month and highest in the ninth.

The distance from peak to peak is roughly 1425-750, or 675 days. A year on Mars appears to be about 675 days from the plot. Internet search confirms that a Mars year is equivalent to 687 earth days.

### Step 6: Save the Data

Export the DataFrame to a CSV file.

In [ ]:
# Write the data to a CSV
df.to_csv('mars_data.csv', index=False)

In [ ]:
browser.quit()